In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds

print(tf.__version__)

import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from matplotlib.colors import ListedColormap
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, classification_report, accuracy_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
# plot_confusion_matrix
from nltk.tokenize import word_tokenize
import scipy.sparse as sp

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-06-29 13:27:47--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.08s   

2023-06-29 13:27:47 (4.04 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-06-29 13:27:47--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [7]:
df_train = pd.read_csv('train-data.tsv', delimiter='\t')
df_test = pd.read_csv('valid-data.tsv', delimiter='\t')
df_train.columns = ['spam/ham', 'message']
df_test.columns = ['spam/ham', 'message']

In [41]:
pd.DataFrame(df_train)

,spam/ham,message,No_of_Characters,No_of_Words,No_of_sentence
0,ham,you can never do nothing,24,5,1
1,ham,now u sound like manky scouse boy steve like i...,115,23,1
2,ham,mum say we wan to go then go then she can shun...,83,17,1
3,ham,never y lei i v lazy got wat dat day send me d...,77,17,1
4,ham,in xam hall boy asked girl tell me the startin...,185,35,1
...,...,...,...,...,...
4173,ham,just woke up yeesh it late but i didn t fall a...,74,16,1
4174,ham,what do u reckon a need arrange transport if u...,69,15,1
4175,spam,free entry into our weekly competition just te...,106,19,1
4176,spam,pls stop bootydelious f is inviting you to be ...,152,27,1


In [43]:
pd.DataFrame(df_test)

,spam/ham,message,No_of_Characters,No_of_Words,No_of_sentence
0,ham,not much just some textin how bout you,42,8,1
1,ham,i probably won t eat at all today i think i m ...,94,23,1
2,ham,don t give a flying monkey wot they think and ...,103,22,1
3,ham,who are you seeing,19,4,1
4,ham,your opinion about me over jada kusruthi lovab...,136,17,1
...,...,...,...,...,...
1386,ham,true dear i sat to pray evening and felt so so...,75,18,1
1387,ham,what will we do in the shower baby,36,8,1
1388,ham,where are you what are you doing are yuou work...,139,30,1
1389,spam,ur cash balance is currently pound to maximize...,148,26,1


In [9]:
df_train['message'] = df_train['message'].astype(str)
df_test['message'] = df_test['message'].astype(str)

In [25]:
# Adding a column of numbers of charachters,words and sentences in each msg

df_train["No_of_Characters"] = df_train["message"].apply(len)
df_train["No_of_Words"]=df_train.apply(lambda row: nltk.word_tokenize(row["message"]), axis=1).apply(len)
df_train["No_of_sentence"]=df_train.apply(lambda row: nltk.sent_tokenize(row["message"]), axis=1).apply(len)

df_test["No_of_Characters"] = df_test["message"].apply(len)
df_test["No_of_Words"]=df_test.apply(lambda row: nltk.word_tokenize(row["message"]), axis=1).apply(len)
df_test["No_of_sentence"]=df_test.apply(lambda row: nltk.sent_tokenize(row["message"]), axis=1).apply(len)

df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
No_of_Characters,4178.0,80.458114,60.285037,2.0,35.0,62.0,122.0,910.0
No_of_Words,4178.0,15.727621,11.524786,0.0,7.0,13.0,23.0,190.0
No_of_sentence,4178.0,0.999521,0.021877,0.0,1.0,1.0,1.0,1.0


In [26]:
def clean_msg(text):
  remove_non_alpha = re.sub('[^a-zA-Z]', ' ', text)
  remove_non_alpha = remove_non_alpha.lower()
  return remove_non_alpha


df_train['message'] = df_train['message'].apply(clean_msg)
df_test['message'] = df_test['message'].apply(clean_msg)

In [27]:
pd.DataFrame(df_train)

,spam/ham,message,No_of_Characters,No_of_Words,No_of_sentence
0,ham,you can never do nothing,24,5,1
1,ham,now u sound like manky scouse boy steve like ...,115,23,1
2,ham,mum say we wan to go then go then she can s...,83,17,1
3,ham,never y lei i v lazy got wat dat day ...,77,17,1
4,ham,in xam hall boy asked girl tell me the startin...,185,35,1
...,...,...,...,...,...
4173,ham,just woke up yeesh its late but i didn t fal...,74,16,1
4174,ham,what do u reckon as need arrange transport i...,69,15,1
4175,spam,free entry into our weekly competition ju...,106,19,1
4176,spam,pls stop bootydelious f is inviting you ...,152,27,1


In [28]:
pd.DataFrame(df_test)

,spam/ham,message,No_of_Characters,No_of_Words,No_of_sentence
0,ham,not much just some textin how bout you,42,8,1
1,ham,i probably won t eat at all today i think i m...,94,23,1
2,ham,don t give a flying monkeys wot they think and...,103,22,1
3,ham,who are you seeing,19,4,1
4,ham,your opinion about me over jada kusr...,136,17,1
...,...,...,...,...,...
1386,ham,true dear i sat to pray evening and felt so s...,75,18,1
1387,ham,what will we do in the shower baby,36,8,1
1388,ham,where are you what are you doing are yuou ...,139,30,1
1389,spam,ur cash balance is currently pounds to m...,148,26,1


In [29]:
type(df_train['message'][0])

str

In [15]:
# Tokenization

# df_train['message'] = df_train['message'].apply(word_tokenize)
# df_test['message'] = df_test['message'].apply(word_tokenize)

In [16]:
# Stop Word Removal

# stop_words = set(stopwords.words('english'))

# def remove_stopwords(word_list):
#     filtered_words = []
#     for word in word_list:
#         if word not in stop_words:
#             filtered_words.append(word)
#     return filtered_words

# df_train['message'].apply(remove_stopwords)
# df_test['message'].apply(remove_stopwords)

In [30]:
# lemmatization does Tokenization, Stop Word Removal,
# removing any irrelevant characters, special symbols, or punctuation marks

lemmatizer = WordNetLemmatizer()

def split_sentence(sentence):
  return sentence.split()


def preprocess_text(word_list):

    filtered_words = []
    for word in word_list:
      lem_word = lemmatizer.lemmatize(word)
      filtered_words.append(lem_word)
    return filtered_words

# def list_to_str(sentence):
#     sentence_str = ''
#     for word in sentence:
#         sentence_str += word + ' '
#     return sentence_str[:-1]

def list_to_str(sentence):
    sentence_str = ' '.join(sentence)
    return sentence_str

In [31]:
pd.DataFrame(df_train)

,spam/ham,message,No_of_Characters,No_of_Words,No_of_sentence
0,ham,you can never do nothing,24,5,1
1,ham,now u sound like manky scouse boy steve like ...,115,23,1
2,ham,mum say we wan to go then go then she can s...,83,17,1
3,ham,never y lei i v lazy got wat dat day ...,77,17,1
4,ham,in xam hall boy asked girl tell me the startin...,185,35,1
...,...,...,...,...,...
4173,ham,just woke up yeesh its late but i didn t fal...,74,16,1
4174,ham,what do u reckon as need arrange transport i...,69,15,1
4175,spam,free entry into our weekly competition ju...,106,19,1
4176,spam,pls stop bootydelious f is inviting you ...,152,27,1


In [32]:
pd.DataFrame(df_test)

,spam/ham,message,No_of_Characters,No_of_Words,No_of_sentence
0,ham,not much just some textin how bout you,42,8,1
1,ham,i probably won t eat at all today i think i m...,94,23,1
2,ham,don t give a flying monkeys wot they think and...,103,22,1
3,ham,who are you seeing,19,4,1
4,ham,your opinion about me over jada kusr...,136,17,1
...,...,...,...,...,...
1386,ham,true dear i sat to pray evening and felt so s...,75,18,1
1387,ham,what will we do in the shower baby,36,8,1
1388,ham,where are you what are you doing are yuou ...,139,30,1
1389,spam,ur cash balance is currently pounds to m...,148,26,1


In [34]:
vectorizer = TfidfVectorizer()

df_train['message'] = df_train['message'].apply(split_sentence).apply(preprocess_text).apply(list_to_str)
df_test['message'] = df_test['message'].apply(split_sentence).apply(preprocess_text).apply(list_to_str)


X_train = vectorizer.fit_transform(df_train['message'])
X_test = vectorizer.transform(df_test['message'])


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train['spam/ham'])
y_test = label_encoder.transform(df_test['spam/ham'])


# Get the feature names (tokens)
# feature_names = vectorizer.get_feature_names_out(X_train)
# Print the feature names
# print(feature_names)

In [35]:
print(pd.DataFrame(X_train))
print(pd.DataFrame(X_test))

                                                      0
0       (0, 3601)\t0.5938671832447373\n  (0, 1435)\t...
1       (0, 1699)\t0.22296207740775956\n  (0, 1403)\...
2       (0, 1741)\t0.3415098378367859\n  (0, 2138)\t...
3       (0, 3695)\t0.2219308999344188\n  (0, 6036)\t...
4       (0, 2656)\t0.2417611550648996\n  (0, 4009)\t...
...                                                 ...
4173    (0, 6125)\t0.40884703331957734\n  (0, 6020)\...
4174    (0, 4319)\t0.44192159237626966\n  (0, 5539)\...
4175    (0, 5607)\t0.37932504297080955\n  (0, 1039)\...
4176    (0, 613)\t0.5448907104366667\n  (0, 2009)\t0...
4177    (0, 1976)\t0.2781953700438588\n  (0, 4416)\t...

[4178 rows x 1 columns]
                                                      0
0       (0, 6151)\t0.19012774189544865\n  (0, 4913)\...
1       (0, 6155)\t0.1593457458932825\n  (0, 6024)\t...
2       (0, 6050)\t0.29585381711258446\n  (0, 5383)\...
3       (0, 6151)\t0.2363294605283802\n  (0, 5951)\t...
4       (0, 6155)\t0.12

In [36]:
#Testing on the following classifiers
classifiers = [MultinomialNB(),
               RandomForestClassifier(),
               KNeighborsClassifier(),
               SVC()]
for cls in classifiers:
    cls.fit(X_train, y_train)

# Dictionary of pipelines and model types for ease of reference
pipe_dict = {0: "NaiveBayes", 1: "RandomForest", 2: "KNeighbours",3: "SVC"}

In [37]:
# Cossvalidation
for i, model in enumerate(classifiers):
  cv_scores = cross_val_score(model, X_train, y_train, scoring="accuracy", cv=10)
  avg_score = cv_scores.mean()
  print(f"{pipe_dict[i]},  {avg_score}")

NaiveBayes,  0.958354273518984
RandomForest,  0.9760679494681765
KNeighbours,  0.9128762062120639
SVC,  0.9763071839179375


In [38]:
pred_y = classifiers[3].predict(X_test)
# pred_y = pred_y.reshape(-1, 1)
# y_test = y_test.reshape(-1, 1)
# pred_y_series = pd.Series(pred_y)
# pred_y_sparse = vectorizer.transform(pred_y)
# pred_y_transformed = vectorizer.transform(pd.Series(pred_y))
# pred_y_series = pd.Series(pred_y)
# pred_y_sparse = sp.csr_matrix(pred_y)
# test_accuracy = classifiers[3].score(y_test, pred_y_series)
test_accuracy = accuracy_score(y_test, pred_y)
test_accuracy

0.9849029475197699

In [39]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):

  processed_pred_text = pd.Series(pred_text).apply(split_sentence).apply(preprocess_text).apply(list_to_str)

  processed_pred_text_vectorized = vectorizer.transform(processed_pred_text)

  pred_label = classifiers[3].predict(processed_pred_text_vectorized)
  # pred_label = classifiers[3].predict(processed_pred_text_vectorized)[0]

  pred_label_text = label_encoder.inverse_transform(pd.Series(pred_label))[0]
  # pred_label_text = label_encoder.inverse_transform([pred_label])

  prediction = [float(pred_label), str(pred_label_text)]

  return (prediction)

pred_text = "how are you doing today?"
pred_text2 = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

print(predict_message(pred_text2))

[0.0, 'ham']
[1.0, 'spam']


In [40]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
